In [45]:
import os
import numpy as np
import pandas as pd
import re

df_target = pd.read_csv("y-enjoyment.csv")

X = []
y = []
person_ids = []
for file in os.listdir("text-embeddings-short"):
    if file.endswith("csv"):
        # search for number in filename
        match = re.search(r"(\d+)", file)
        if match is None:
            continue
        person_id = int(match.group(1))
        person_ids.append(person_id)
        df = pd.read_csv(
            os.path.join("text-embeddings-short", file), header=0
        )
        X_aux = df.iloc[:, 1:].values
        X_aux = np.mean(X_aux, axis=0)
        X.append(X_aux)
        y.append(df_target[df_target["user_id"] == person_id]["Average"])

# sort by person_id
sorted_indices = np.argsort(person_ids)
X = [X[i] for i in sorted_indices]
y = [y[i] for i in sorted_indices]
person_ids = [person_ids[i] for i in sorted_indices]
# convert to numpy arrays
X = np.vstack(X)
y = np.vstack(y)
print(X.shape)
print(y.shape)

(38, 4096)
(38, 1)


In [46]:
import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import Ridge, LinearRegression # Or RidgeCV for automatic alpha tuning
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr
from sklearn.utils import shuffle
#import XGBoost
from sklearn.ensemble import GradientBoostingRegressor



# Initialize LeaveOneOut
loo = LeaveOneOut()

# Lists to store actual values and predictions from each fold
y_true_all = []
y_pred_all = []

# --- Choose your Ridge approach ---
# Option 1: Fixed Alpha
#ridge_alpha = 1.0 # Choose an appropriate alpha
#model_to_use = Ridge(alpha=ridge_alpha)

# Option 2: Tuned Alpha using RidgeCV (performs internal CV to find best alpha)
# Note: RidgeCV fits on the whole train split using the found best alpha.
# If you need alpha tuned PER fold for the LOOCV prediction, you'd need a nested loop.
# For simplicity, let's stick to fixed alpha here based on the request structure.


print(f"Starting LOOCV for {len(y)} samples...")

# Assuming X and y are your input data and target values
X, y, person_ids = shuffle(X, y, person_ids) # Shuffle data with a fixed random seed

# Loop through LOOCV splits
for train_index, test_index in loo.split(X):

    model_to_use = LinearRegression() # cv=5 for inner loop tuning

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the Ridge model
    model_to_use.fit(X_train, y_train)

    # Predict on the left-out sample
    y_pred = model_to_use.predict(X_test)

    # Store the actual and predicted values
    y_true_all.append(y_test[0])
    y_pred_all.append(y_pred[0])

print("LOOCV finished.")

# Convert lists to numpy arrays for metric calculation
y_true_all = np.array(y_true_all).flatten()
y_pred_all = np.array(y_pred_all).flatten()

# Calculate final metrics based on aggregated predictions
mse = mean_squared_error(y_true_all, y_pred_all)
r2 = r2_score(y_true_all, y_pred_all)
correlation, p_value = pearsonr(y_true_all, y_pred_all)

print(f"\n--- LOOCV Results (N={len(y_true_all)}) ---")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"R-squared (R2): {r2:.4f}")
print(f"Pearson Correlation: {correlation:.4f}")
print(f"P-value for Correlation: {p_value:.4f}")

# Save the predictions to a CSV file
predictions_df = pd.DataFrame({
    'Person ID': person_ids,
    'True Values': y_true_all,
    'Predicted Values': y_pred_all.round(2),
})
# Order by Person ID
predictions_df.sort_values(by='Person ID', inplace=True)
modality_suffix = "text"
predictions_df.to_csv(f'predictions{modality_suffix}.csv', index=False)

Starting LOOCV for 38 samples...
LOOCV finished.

--- LOOCV Results (N=38) ---
Mean Squared Error (MSE): 1.2872
R-squared (R2): 0.3111
Pearson Correlation: 0.5877
P-value for Correlation: 0.0001


In [51]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import pearsonr
import pandas as pd

# ------------------- 1. Load Your Data -------------------
# Replace the CSV paths with your actual files
predictions_audio_np = pd.read_csv("predictions-audio-attention.csv")["Predicted Values"].values
true_values_np       = pd.read_csv("predictions-audio-attention.csv")["True Values"].values
predictions_text_np  = pd.read_csv("predictions-text-attention.csv")["Predicted Values"].values

# Ensure they are NumPy arrays
true_values_np       = np.array(true_values_np)
predictions_audio_np = np.array(predictions_audio_np)
predictions_text_np = np.array(predictions_text_np)

# Check if lengths match
assert len(true_values_np) == len(predictions_audio_np) == len(predictions_text_np), \
       "Error: True values and all prediction arrays must have the same length."

all_predictions = [predictions_audio_np, predictions_text_np]

# ------------------- 2. Evaluation Function (with MAE) -------------------
def evaluate_predictions(true_vals, pred_vals, model_name="Model"):
    """Calculates and prints R2, MSE, MAE, Pearson Correlation, and p-value."""
    # Correlation
    if len(true_vals) < 2:
        print(f"Warning: Need at least 2 data points for correlation for {model_name}.")
        corr, p_value = np.nan, np.nan
    else:
        corr, p_value = pearsonr(true_vals, pred_vals)

    # Metrics
    r2  = r2_score(true_vals, pred_vals)
    mse = mean_squared_error(true_vals, pred_vals)
    mae = mean_absolute_error(true_vals, pred_vals)

    # Output
    print(f"\n--- Evaluation Results for: {model_name} ---")
    print(f"R-squared (R2):               {r2:.4f}")
    print(f"Mean Squared Error (MSE):     {mse:.4f}")
    print(f"Mean Absolute Error (MAE):    {mae:.4f}")
    print(f"Pearson Correlation:          {corr:.4f}")
    print(f"P-value (for Correlation):    {p_value:.4f}")
    if not np.isnan(p_value):
        if p_value < 0.05:
            print("  (Correlation is statistically significant at p < 0.05)")
        else:
            print("  (Correlation is not statistically significant at p < 0.05)")
    print("---------------------------------------------")

# ------------------- 3. Run Evaluation -------------------
print(f"\nEvaluating {len(true_values_np)} data points.")

# Audio-only
evaluate_predictions(true_values_np, predictions_audio_np, "Audio Only - Attention-based Pooling")

# Text-only
evaluate_predictions(true_values_np, predictions_text_np,    "Text Only - Attention-based Pooling")

# Simple average fusion
fused_predictions_avg_np = np.average(all_predictions, axis=0, weights=[0.6, 0.4])
evaluate_predictions(true_values_np, fused_predictions_avg_np, "Fused (Simple Average)")

# Save fused predictions
fused_predictions_avg_df = pd.DataFrame({
    'True Values': true_values_np,
    'Fused Predictions (Average)': np.round(fused_predictions_avg_np, 2)
})
fused_predictions_avg_df.to_csv('fused_predictions_avg.csv', index=False)

print("\nFused predictions saved to 'fused_predictions_avg.csv'.")



Evaluating 38 data points.

--- Evaluation Results for: Audio Only - Attention-based Pooling ---
R-squared (R2):               0.4047
Mean Squared Error (MSE):     1.1136
Mean Absolute Error (MAE):    0.9161
Pearson Correlation:          0.6399
P-value (for Correlation):    0.0000
  (Correlation is statistically significant at p < 0.05)
---------------------------------------------

--- Evaluation Results for: Text Only - Attention-based Pooling ---
R-squared (R2):               0.3155
Mean Squared Error (MSE):     1.2805
Mean Absolute Error (MAE):    0.9392
Pearson Correlation:          0.5824
P-value (for Correlation):    0.0001
  (Correlation is statistically significant at p < 0.05)
---------------------------------------------

--- Evaluation Results for: Fused (Simple Average) ---
R-squared (R2):               0.4516
Mean Squared Error (MSE):     1.0258
Mean Absolute Error (MAE):    0.8673
Pearson Correlation:          0.6726
P-value (for Correlation):    0.0000
  (Correlation i